In [ ]:
import torch
import cv2
import numpy as np
import torch.nn as nn
import dlib

In [ ]:
class Eye:
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

    def extractor_vid(frame: cv2.typing.MatLike):
        left_eyes = []
        right_eyes = []
        image_gs = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = Eye.detector(image_gs)
        for face in faces:
            landmarks = Eye.predictor(image_gs, face)
            # Left Eye
            min_x, min_y, max_x, max_y = float("inf"), float("inf"), 0, 0
            for n in range(36, 42):
                x = landmarks.part(n).x
                y = landmarks.part(n).y
                min_x = min(min_x, x)
                min_y = min(min_y, y)
                max_x = max(max_x, x)
                max_y = max(max_y, y)
            left_eye = frame[min_y - 20 : max_y + 20, min_x - 20 : max_x + 20, :]
            left_eyes.append(left_eye)

            # Right Eye
            min_x, min_y, max_x, max_y = float("inf"), float("inf"), 0, 0
            for n in range(42, 48):
                x = landmarks.part(n).x
                y = landmarks.part(n).y
                min_x = min(min_x, x)
                min_y = min(min_y, y)
                max_x = max(max_x, x)
                max_y = max(max_y, y)
            right_eye = frame[min_y - 20 : max_y + 20, min_x - 20 : max_x + 20, :]
            right_eyes.append(right_eye)
        return left_eyes, right_eyes

    def extractor_img(image: np.ndarray):
        eyes = Eye.extractor_vid(image)
        eyes=[eyes[i][0] for i in range(len(eyes)) if len(eyes[i])>0]
        return eyes

In [ ]:
# Load the model
class EyeClassifier(nn.Module):
    def __init__(self):
        super(EyeClassifier, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(16, 32, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(64 * 8 * 8, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(-1, 64 * 8 * 8)
        x = self.classifier(x)
        return x

model = EyeClassifier()
model.load_state_dict(torch.load("eye_classifier.pth"))
model

In [ ]:
import collections

cap = cv2.VideoCapture(0)
EYE_COUNT = 30
recent_predictions = collections.deque(maxlen=EYE_COUNT)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    eyes = Eye.extractor_img(frame)
    for eye in eyes:
        if eye.shape[-1] == 3 and eye.shape[0] > 0 and eye.shape[1] > 0 and eye.shape[2] > 0:
            eye = cv2.resize(eye, (64, 64))
            eye = torch.tensor(eye, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0) / 255.0
            pred = model(eye)
            recent_predictions.append(pred.item())
            avg_pred = sum(recent_predictions) / len(recent_predictions)
            if avg_pred > 0.5:
                cv2.putText(frame, "Fatigue", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            else:
                cv2.putText(frame, "Active", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Eye Classification", frame)
    if cv2.waitKey(1) == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()